## 공공데이터 포털 코로나 정보 불러오기
공공데이터 포털에서 검색한 코로나 정보 openapi를 불러온다
openapi에 저장된 인증키와 url 코드를 불러옴

In [1]:
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus
import urllib
import requests
import pandas as pd
import xmltodict
import json
import datetime

today = datetime.datetime.today()
base_date = today.strftime("%Y%m%d")
base_date1 = int(base_date)

key='U%2Bue3nwitwKpRE26tUkRgTj5%2BD2bknqcqqsDwm35y%2B3YhGcrEA%2FSYmTJn1CyrxKcPHdd5sJWJIY8PAV%2BwJxQGg%3D%3D'
url = f'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson?serviceKey=U%2Bue3nwitwKpRE26tUkRgTj5%2BD2bknqcqqsDwm35y%2B3YhGcrEA%2FSYmTJn1CyrxKcPHdd5sJWJIY8PAV%2BwJxQGg%3D%3D&'
queryParams = urlencode({ quote_plus('pageNo') : 1, 
                          quote_plus('numOfRows') : 10,
                          quote_plus('startCreateDt') : base_date1,
                          quote_plus('endCreateDt') : base_date1})

## 날짜 선정

In [2]:
base_date1

20211216

## 데이터 타입 지정

In [3]:
type(base_date1)

int

## url 코드에 지정된 코드값 확인

In [4]:
response = urlopen(url + queryParams) 
results = response.read().decode("utf-8")

In [5]:
url2 = url + queryParams
response = urlopen(url2)

In [6]:
results = response.read().decode("utf-8")
results_to_json = xmltodict.parse(results)
data = json.loads(json.dumps(results_to_json))
print(type(data))   # dic
print(data) 

<class 'dict'>
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': {'item': {'accExamCnt': '18381569', 'createDt': '2021-12-16 09:09:11.111', 'deathCnt': '4518', 'decideCnt': '544117', 'seq': '729', 'stateDt': '20211216', 'stateTime': '00:00', 'updateDt': 'null'}}, 'numOfRows': '10', 'pageNo': '1', 'totalCount': '1'}}}


## 날짜, 확진자 수, 사망자 수 따로 저장

In [7]:
stateDt = (data["response"]["body"]["items"]['item']["stateDt"]) #날짜
stateDt

'20211216'

In [8]:
decideCnt = (data["response"]["body"]["items"]['item']["decideCnt"]) #확진자수
decideCnt

'544117'

In [9]:
deathCnt = (data["response"]["body"]["items"]['item']["deathCnt"]) #사망자수
deathCnt 

'4518'

## 카카오톡 openAPI 토큰 받기

In [12]:
import json
import kakao_utils

KAKAO_TOKEN_FILENAME = "C:/Users/gram/Desktop/파이썬메시업프로젝트/res/kakao_message/kakao_token.json"
KAKAO_APP_KEY = "ec2080a0764544d8c438ffa77c95bd41"
kakao_utils.update_tokens(KAKAO_APP_KEY, KAKAO_TOKEN_FILENAME)

{'access_token': 'YW7CoLUp8aFiEAcp-3BpTQqvrkypMq25mvHZTQorDNMAAAF9wPebaw', 'token_type': 'bearer', 'expires_in': 21599}


{'access_token': 'YW7CoLUp8aFiEAcp-3BpTQqvrkypMq25mvHZTQorDNMAAAF9wPebaw',
 'token_type': 'bearer',
 'refresh_token': 'dKV5wI9lSzPoSlbhoLeVcx-rs3h6pHdctp40CQo9cusAAAF9wPHxAA',
 'expires_in': 21599,
 'scope': 'account_email profile_image talk_message profile_nickname',
 'refresh_token_expires_in': 5183999}

## 정보 발송

In [13]:
corona_url = "http://ncov.mohw.go.kr"
# 날씨 정보 만들기

text =  f"""\
#날짜 : {stateDt} \n
누적확진자 : {decideCnt} \n
사망자 수 : {deathCnt} \n
"""

# 텍스트 템플릿 형식 만들기
template = {
    "object_type": "text",
    "text": text,
    "link": {
        "web_url": corona_url,
        "mobile_web_url": corona_url
    },
    "button_title": "코로나 정보 상세보기"
}

# 카카오톡 메시지 전송

res = kakao_utils.send_message(KAKAO_TOKEN_FILENAME, template)
if res.json().get('result_code') == 0:
    print('코로나 정보 성공적으로 보냈습니다.')
else:
    print('코로나 정보 성공적으로 보내지 못했습니다. 오류메시지 : ', res.json())


코로나 정보 성공적으로 보냈습니다.
